Diagnostics for NR

In [1]:
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import gw_eccentricity
from gw_eccentricity import get_available_methods
from gw_eccentricity import measure_eccentricity
from gw_eccentricity.load_data import load_waveform
from gw_eccentricity.plot_settings import colorsDict, lstyles, lwidths

%load_ext autoreload
%autoreload 2

/home/arif/miniconda3/envs/ecc/lib/python3.9/site-packages/gwtools/rotations.py:63: UserWarning: Could not import GWFrames, needed for rotations module
  _warnings.warn("Could not import GWFrames, needed for rotations module")
/home/arif/miniconda3/envs/ecc/lib/python3.9/site-packages/gwtools/__init__.py:11: UserWarning: Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.
  _warnings.warn("Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.")


In [2]:
import glob
import numpy as np
from tqdm import tqdm
from gw_eccentricity.load_data import load_lvcnr_hack

In [3]:
sxs_nr_waveforms = glob.glob("../data/ecc_waveforms/Non-Precessing/SXS/BBH_S*.h5")

In [4]:
problematic_cases = [8, # need to increase flow and width 
                     34, # Takes too long to load, did not load within tested time.
                     37, # loads too short waveform
                    ]

In [5]:
num_orbits_dict= {"BBH_SKS_q1_e09_D130_DoNotControlLEqualsOneShape_AdjustSphereCExtents_Res3.h5": 3,
                 "BBH_SKS_q1_e09_D130_SKS_DoNotControlLEqualsOneShape_Res3.h5": 3}

In [103]:
def MakeDiagnosticPlots(methods, nr_waveforms, num_orbits_dict, two_column=True):
    """loop through all NR waveforms, and make diagnostics plots 
         methods - list of methods to apply
         nr_waveforms - list of paths to NR waveforms to apply methods to
    """

    # make output objects, one per method
    pps = [PdfPages(f"NR-diagnostics-{method}.pdf") for method in methods]


    comparisons = PdfPages("NR_diagnostics-comparisons.pdf")

    for idx, waveform_path in enumerate(nr_waveforms):
        w = waveform_path.split("/")[-1]
        print(f"===== {idx}/{len(nr_waveforms)} ===== {w} =====", flush=True)
        kwargs = {"filepath": waveform_path,
                "include_zero_ecc": True,
                "num_orbits_to_remove_as_junk": num_orbits_dict.get(w, 2)}
        dataDict = load_lvcnr_hack(**kwargs)
        tref_vec = -2000 # dataDict["t"]


        # for ecc-comparison plot
        fig0, ax0 = plt.subplots(figsize=(12, 6))

        for method,pp in zip(methods, pps):
            tref_out, ecc_vec, mean_ano_vec, eccMethod = measure_eccentricity(
                tref_in=tref_vec,
                dataDict=dataDict,
                method=method, 
                return_gwecc_object=True,
                extra_kwargs={"debug": False})
            fig, ax = eccMethod.make_diagnostic_plots(journal="APS", two_columns=two_column)
            q = dataDict["params_dict"]["q"]
            ecc = dataDict["params_dict"]["ecc"]
            chi1 = np.around(dataDict["params_dict"]["chi1"], 5)
            chi2 = np.around(dataDict["params_dict"]["chi2"], 5)
            title = (f"case {idx}/{len(sxs_nr_waveforms)-1}, "
                    f"$q$={q:.1f}, "
                    f"$\chi_{1}$={chi1}, "
                    f"$\chi_{2}$={chi2}, "
                    f"ecc={ecc:.2f}\n"
                    f"method={method}, "
                    f"id={w}")
            if two_column:
                fig.suptitle(title, ha="center", fontsize=10)
            else:
                ax[0].set_title(title, ha="center", fontsize=10)
            fig.set_tight_layout(True)
            fig.savefig(pp,format='pdf')
            plt.close(fig)

            eccMethod.plot_measured_ecc(fig0, ax0, **{"label": method, "lw": lwidths[method], "ls": lstyles[method], "c": colorsDict[method]})
        ax0.set_title(f"q={q:.1f}, ecc={ecc:.2f}, {w}", fontsize=14)
        ax0.legend();
        fig0.savefig(comparisons,format='pdf')
        plt.close(fig0)

    for pp in pps:
        pp.close()
    comparisons.close()


In [105]:
# to test, analyse only two waveforms:

MakeDiagnosticPlots(gw_eccentricity.get_available_methods(), sxs_nr_waveforms[30:31], num_orbits_dict, two_column=True)

===== 0/1 ===== BBH_SHK_q6_e07_D45_Res3.h5 =====


In [ ]:
# ALL SXS-WAVEFORMS.  THIS TAKES A FEW MINUTES !!!

MakeDiagnosticPlots(gw_eccentricity.get_available_methods(), sxs_nr_waveforms, num_orbits_dict)

### Loop over ET waveforms

In [ ]:
et_waveforms = sorted(glob.glob("../data/ecc_waveforms/Non-Precessing/ET/*.h5"))

### Using ResidualAmplitude

In [ ]:
method = "ResidualAmplitude"
for idx, waveform_path in tqdm(enumerate(et_waveforms)):
    if idx in [5, 6]: # too few cycles
        continue
    kwargs = {"filepath": waveform_path,
             "include_zero_ecc": True,
             "num_orbits_to_remove_as_junk": 0}
    dataDict = load_lvcnr_hack(**kwargs)
    tref_vec = dataDict["t"]
    tref_out, ecc_vec, mean_ano_vec, eccMethod = measure_eccentricity(
        tref_in=tref_vec,
        dataDict=dataDict,
        method=method, 
        return_ecc_method=True,
        extra_kwargs={"debug": False})
    fig, ax = eccMethod.make_diagnostic_plots()
    w = waveform_path.split("/")[-1]
    q = dataDict["params_dict"]["q"]
    ecc = dataDict["params_dict"]["ecc"]
    chi1 = dataDict["params_dict"]["chi1"]
    chi2 = dataDict["params_dict"]["chi2"]
    ax[0].set_title(f"===============================================================================\n"
                f"case {idx}/{len(sxs_nr_waveforms)-1}\n"
                f"q={q:.1f}\n"
                f"chi1={chi1}\n"
                f"chi2={chi2}\n"
                f"ecc={ecc:.2f}\n"
                f"method={method}\n"
                f"id={w}\n"
                "===============================================================================", fontsize=14)

In [ ]:
kwargs = {"filepath": et_waveforms[1],
          "include_zero_ecc": True,
          "num_orbits_to_remove_as_junk": 0}
dataDict = load_NR(**kwargs)

In [ ]:
import numpy as np
plt.plot(dataDict["t"], np.abs(dataDict["hlm"][(2, 2)]))

### With amplitude

In [ ]:
method = "Amplitude"
for idx, waveform_path in tqdm(enumerate(et_waveforms)):
    if idx in [1, 5, 6]: # too few cycles
        continue
    kwargs = {"filepath": waveform_path,
             "include_zero_ecc": True,
             "num_orbits_to_remove_as_junk": 0}
    dataDict = load_lvcnr_hack(**kwargs)
    tref_vec = dataDict["t"]
    tref_out, ecc_vec, mean_ano_vec, eccMethod = measure_eccentricity(
        tref_in=tref_vec,
        dataDict=dataDict,
        method=method, 
        return_ecc_method=True,
        extra_kwargs={"debug": False})
    fig, ax = eccMethod.make_diagnostic_plots()
    w = waveform_path.split("/")[-1]
    q = dataDict["params_dict"]["q"]
    ecc = dataDict["params_dict"]["ecc"]
    chi1 = dataDict["params_dict"]["chi1"]
    chi2 = dataDict["params_dict"]["chi2"]
    ax[0].set_title(f"===============================================================================\n"
                f"case {idx}/{len(sxs_nr_waveforms)-1}\n"
                f"q={q:.1f}\n"
                f"chi1={chi1}\n"
                f"chi2={chi2}\n"
                f"ecc={ecc:.2f}\n"
                f"method={method}\n"
                f"id={w}\n"
                "===============================================================================", fontsize=14)

### FrequencyFits

In [ ]:
method = "FrequencyFits"
for idx, waveform_path in tqdm(enumerate(et_waveforms)):
    if idx in [0, 1, 2, 4, 5]: # too few cycles
        continue
    kwargs = {"filepath": waveform_path,
             "include_zero_ecc": True,
             "num_orbits_to_remove_as_junk": 0}
    dataDict = load_lvcnr_hack(**kwargs)
    tref_vec = dataDict["t"]
    tref_out, ecc_vec, mean_ano_vec, eccMethod = measure_eccentricity(
        tref_in=tref_vec,
        dataDict=dataDict,
        method=method, 
        return_ecc_method=True,
        extra_kwargs={"debug": False})
    fig, ax = eccMethod.make_diagnostic_plots()
    w = waveform_path.split("/")[-1]
    q = dataDict["params_dict"]["q"]
    ecc = dataDict["params_dict"]["ecc"]
    chi1 = dataDict["params_dict"]["chi1"]
    chi2 = dataDict["params_dict"]["chi2"]
    ax[0].set_title(f"===============================================================================\n"
                f"case {idx}/{len(sxs_nr_waveforms)-1}\n"
                f"q={q:.1f}\n"
                f"chi1={chi1}\n"
                f"chi2={chi2}\n"
                f"ecc={ecc:.2f}\n"
                f"method={method}\n"
                f"id={w}\n"
                "===============================================================================", fontsize=14)

# check peaks and troughs appear alternatively

In [ ]:
method = "ResidualAmplitude"
waveform_path = sxs_nr_waveforms[32]
kwargs = {"filepath": waveform_path,
         "include_zero_ecc": True,
         "num_orbits_to_remove_as_junk": 3}
dataDict = load_lvcnr_hack(**kwargs)
tref_vec = dataDict["t"]
tref_out, ecc_vec, mean_ano_vec, eccMethod = measure_eccentricity(tref_in=tref_vec, dataDict=dataDict, method=method, 
                                                                return_ecc_method=True,
                                                                extra_kwargs={"debug": False,
                                                                             #"extrema_finding_kwargs": {"width": 500}
                                                                             })
fig, ax = eccMethod.make_diagnostic_plots()
w = waveform_path.split("/")[-1]
q = dataDict["params_dict"]["q"]
ecc = dataDict["params_dict"]["ecc"]
chi1 = dataDict["params_dict"]["chi1"]
chi2 = dataDict["params_dict"]["chi2"]

ax[0].set_title(f"===============================================================================\n"
                f"case 1/61\n"
                f"q={q:.1f}\n"
                f"chi1={chi1}\n"
                f"chi2={chi2}\n"
                f"ecc={ecc:.2f}\n"
                f"method={method}\n"
                f"id={w}\n"
                "===============================================================================", fontsize=14)

In [ ]:
eccMethod.peaks_location

In [ ]:
eccMethod.troughs_location

In [ ]:
len(eccMethod.peaks_location), len(eccMethod.troughs_location)

In [ ]:
for idx in range(len(eccMethod.peaks_location)):
    print(idx, (eccMethod.peaks_location[idx] - eccMethod.troughs_location[idx])
              * (eccMethod.troughs_location[idx + 1] - eccMethod.peaks_location[idx]))

In [ ]:
eccMethod.peaks_location[12], eccMethod.troughs_location[12]

In [ ]:
eccMethod.troughs_location[12 + 1]